In [1]:
import torch
import numpy as np
import scipy as sp
import sys
sys.path.append('/Users/judd/Documents/optimisation_of_superconduncting_circuits/core')
from zeropi import ZeroPi
import general as general
import scqubits as sc


In [15]:
#Computing T2
EJ = torch.tensor(10.00, requires_grad=True, dtype=torch.double)
EL = torch.tensor(10.00, requires_grad=True, dtype=torch.double)
ECJ = torch.tensor(0.5, requires_grad=True, dtype=torch.double)
EC = torch.tensor(2.5, requires_grad=True, dtype=torch.double)
dEJ = torch.tensor(2.5, requires_grad=True, dtype=torch.double)
dCJ = torch.tensor(2.5, requires_grad=True, dtype=torch.double)
flux = torch.tensor(0.5, requires_grad=True, dtype=torch.double)

ng = 0.1
ncut = 30
truncated_dim = 10
pt_count = 10
min_val = -6*np.pi
max_val = 6*np.pi
hamiltonian_creation = 'auto_H'



zeropi = ZeroPi(EJ=EJ, EL=EL, ECJ=ECJ, EC=EC, dEJ=dEJ, dCJ=dCJ, ng=ng, flux=flux, ncut=ncut, truncated_dim=truncated_dim, pt_count=pt_count, min_val=min_val, max_val=max_val, hamiltonian_creation=hamiltonian_creation)

general.effective_t1_rate(zeropi , "t1_inductive")
general.effective_tphi_rate(zeropi , zeropi.tphi_supported_noise_channels())
general.t2(zeropi , zeropi.t1_supported_noise_channels(), zeropi.tphi_supported_noise_channels())


tensor(10., dtype=torch.float64, requires_grad=True)
tensor(0.5000, dtype=torch.float64, requires_grad=True)


TypeError: transpose() received an invalid combination of arguments - got (), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [16]:
#compare my pytorch computations to scqubits
zeropi_sc = sc.ZeroPi(
            grid = sc.Grid1d(min_val=min_val, max_val=max_val, pt_count=pt_count),
            EJ   = EJ.item(),
            EL   = EL.item(), 
            ECJ  = ECJ.item(),
            EC   = EC.item(),
            ng   = ng,
            flux = flux.item(),
            ncut = ncut, 
            dEJ = dEJ, 
            dCJ = dCJ)

print(zeropi_sc.t1_inductive() - 1/general.effective_t1_rate(zeropi , "t1_inductive").item())
print(zeropi_sc.t1_flux_bias_line() - 1/general.effective_t1_rate(zeropi , "t1_flux_bias_line").item())
print(zeropi_sc.tphi_1_over_f_cc() - 1/general.effective_tphi_rate(zeropi , "tphi_1_over_f_cc").item())
print(zeropi_sc.tphi_1_over_f_flux() - 1/general.effective_tphi_rate(zeropi , "tphi_1_over_f_flux()").item())

See documentation for details.
This warning can be disabled by executing:
scqubits.settings.T1_DEFAULT_WARNING=False

 /Users/judd/Documents/optimisation_of_superconduncting_circuits/venv/lib/python3.8/site-packages/scqubits/core/noise.py: 1195

TypeError: transpose() received an invalid combination of arguments - got (), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)
